#### Install libtorrent

In [ ]:

!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent

#### Initialize Session

In [ ]:
import libtorrent as lt
import time
import requests
import os
from IPython.display import display
import ipywidgets as widgets
from google.colab import drive

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

# === Initialize Torrent Session (faster peer discovery)
ses = lt.session()
ses.listen_on(6881, 6891)
ses.start_dht()
ses.start_lsd()
ses.start_upnp()
ses.start_natpmp()
downloads = []

# === Mount Google Drive
drive.mount("/content/drive")

#### Input Torrent/Magnet Link

In [ ]:
# === Input Torrent/Magnet Link
TORRENT_LINK = input("📥 Enter Torrent/Magnet Link: ").strip()

params = {}
save_path_root = "/content/drive/My Drive/Torrent"

# Check if it's a .torrent file URL (like YTS)
if TORRENT_LINK.endswith(".torrent") or "torrent/download" in TORRENT_LINK:
    print("🔗 Detected .torrent file link, downloading...")
    r = requests.get(TORRENT_LINK)
    torrent_file_path = "/content/temp.torrent"
    with open(torrent_file_path, "wb") as f:
        f.write(r.content)

    ti = lt.torrent_info(torrent_file_path)
    folder_name = ti.name()
    save_path = f"{save_path_root}" #/{folder_name}
    os.makedirs(save_path, exist_ok=True)

    params = {"save_path": save_path, "ti": ti}
    downloads.append(ses.add_torrent(params))

# Otherwise, treat it as a magnet link
else:
    print("🧲 Detected magnet link...")
    folder_name = "Magnet_Download"
    save_path = f"{save_path_root}"  #/{folder_name}
    os.makedirs(save_path, exist_ok=True)

    params = {"save_path": save_path}
    downloads.append(lt.add_magnet_uri(ses, TORRENT_LINK, params))

# === Progress Bar Setup
state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(step=0.01, disabled=True, layout=layout, style=style)
    for _ in downloads
]
display(*download_bars)

# === Download Loop
while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()
            eta = (
                (s.total_wanted - s.total_done) / s.download_rate
                if s.download_rate > 0
                else 0
            )
            eta_str = f"{eta/60:.1f} min" if eta > 0 else "∞"

            bar.description = f"{download.name()} | {s.download_rate/1000:.1f} kB/s | {s.progress*100:.1f}% | ETA: {eta_str} | {state_str[s.state]}"
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close()
            download_bars.remove(bar)
            print(f"✅ {download.name()} complete!")

    time.sleep(1)

#### Auto Unmount Drive

In [ ]:
drive.flush_and_unmount()
print("✅ Google Drive unmounted!")